<a href="https://colab.research.google.com/github/carlos-alves-one/-ML-Zoomcamp-HW-6/blob/main/ML_Zoomcamp_HW_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Machine Learning ZoomCamp - Homework 6
**Author.....: Carlos Manuel de Oliveira Alves**<br>
**Created...: 18/10/2022**